In [1]:
import hashlib
import inspect
import types

In [2]:
def fdigest(filename):
    f = open(filename,mode='rb')
    m = hashlib.sha256(f.read())
    f.close()
    return m.hexdigest()

In [3]:
h = fdigest('Loads.ipynb')
h

'd5b691758babfd8d3ddd6ad3842c08c87c825361fd9eaf8ec904ac0a7faca53d'

In [5]:
def extend(new):
    name = new.__name__
    old = globals().get(name,None)
    if old is None:
        old = type(name,(object,),{})
    ok = ['__init__']
    for a,v in inspect.getmembers(new):
        if not a.startswith('_') or a in ok:
            if type(v) is types.MethodType:
                v = types.MethodType(v.im_func,v.im_self,old)
            elif type(v) is property:
                v = property(v.fget,v.fset,v.fdel)
            elif type(v) is types.FunctionType:
                v = staticmethod(types.FunctionType(v.func_code,v.func_globals,v.func_name,v.func_defaults,v.func_closure))
            setattr(old,a,v)
    return old

In [6]:
class Foo(object):
    def one(self):
        return 1

In [7]:
id(Foo), Foo

(3042161156L, __main__.Foo)

In [8]:
@extend
class Foo:
    def two(self):
        return 2

In [9]:
@extend
class Foo:
    def __init__(s):
        s.__p = 13
    def three(self):
        return 3
    @property
    def p(s):
        return s,s.__p
    @p.setter
    def p(s,v):
        s.__p = v
    @classmethod
    def cm(c):
        return 'class',c
    @staticmethod
    def sm():
        return 77

In [10]:
f = Foo()

In [11]:
f.one()

1

In [12]:
f.two()

2

In [13]:
f.three()

3

In [14]:
f.p

(<__main__.Foo at 0xb553124c>, 13)

In [15]:
f.p = 3686868

In [16]:
f.p

(<__main__.Foo at 0xb553124c>, 3686868)

In [17]:
f.cm()

('class', <class __main__.Foo at 0xb553d20c>)

In [18]:
f.sm()

77

In [19]:
class Zap(object):
    pass

In [20]:
def mc(*l):
    print 'mc:',l
def mc2(*l):
    print 'mc2:',l

@mc2
class Zipp(object):
    __metaclass__ = mc
    A = 1
    def z(s):
        return s

mc: ('Zipp', (<type 'object'>,), {'A': 1, '__module__': '__main__', 'z': <function z at 0xb554202c>, '__metaclass__': <function mc at 0xb5516e64>})
mc2: (None,)
